# image re-labelling code
going over the images i have already labelled, and re-labelling them.

In [2]:
import numpy as np
import pandas as pd
import os
#from PIL import Image
from glob import glob
import csv
import matplotlib.pyplot as plt
from IPython.display import clear_output
import math
import shutil

In [4]:
## directory of set of images i would like to label + corresponding csv to populate
folder_path = '/gws/nopw/j04/dcmex/users/ezriab/image_labelling/2ds_10000_sample/'
# open csv quickly to check it is correct + in the format we expect
csv_file_name = '2ds_10000_8_dif.csv'
saving_csv_name = '2ds_10000_8_dif.csv'
csv_full_path = os.path.join(folder_path,csv_file_name)
full_save_csv_path = os.path.join(folder_path,saving_csv_name)
csv_read_pd = pd.read_csv(csv_full_path)
print(f'reading {csv_full_path}')
print(f'saving to {full_save_csv_path}')
n_images_done = csv_read_pd['2nd_label'].count()
print(f'number of images already labelled: {n_images_done}')
csv_read_pd.head()

reading /gws/nopw/j04/dcmex/users/ezriab/image_labelling/2ds_10000_sample/2ds_10000_8_dif.csv
saving to /gws/nopw/j04/dcmex/users/ezriab/image_labelling/2ds_10000_sample/2ds_10000_8_dif.csv
number of images already labelled: 8619


,image_name,number_label,2nd_label
0,1697685_23ch1,1.0,1.0
1,669294_20ch0,1.0,1.0
2,601343_19ch0,1.0,1.0
3,18058_26ch1,1.0,1.0
4,1261837_25ch0,3.0,NaN


In [3]:
## updating the already labelled csv - make new column + saving in og place
#csv_read_pd['2nd_label'] = np.nan
#csv_read_pd
#full_save_csv_path = os.path.join(folder_path,saving_csv_name)
#csv_read_pd.to_csv(full_save_csv_path, index=False)

## how many of each category
'''
count_cp = csv_read_pd['number_label'].value_counts().get(11.0, 0)
print(count_cp)
'''

"\ncount_cp = csv_read_pd['number_label'].value_counts().get(11.0, 0)\nprint(count_cp)\n"

# label things

In [13]:
number_of_interest = 11 # between 1 to 9, or 11

In [14]:
# okay we are going to need a completly different approach for this
def v2_saving_function(df,save_path):
    df.to_csv(save_path, index=False)
    print(f'{saving_csv_name} saved!')
    return df

# make a list of index where these images are located
interesting_idx = csv_read_pd.index[csv_read_pd['number_label'] == number_of_interest].tolist()

for i in range(len(interesting_idx)):
#for i in range(10):
    idx = interesting_idx[i]
    row = csv_read_pd.loc[idx]
    image_name = row.iloc[0]
    og_label = row.iloc[1]
    new_label = row.iloc[2]

    # I've already labelled it a second time
    if not pd.isna(new_label):
        continue

    ## lets do the whole image labelling thing
    else:
        print(image_name)
        im = Image.open(os.path.join(folder_path, image_name + '.png'))
        plt.figure(figsize=(8, 8))
        plt.imshow(im)
        plt.axis('off')
        plt.show()
        while True: # loop will continue until a valid entry is passed
            inp = input("Enter a label (0-9 or 11, or 'stop' to quit): ") ## label of image

            ## if write 'stop' will stop labelling + save what has been done
            if inp.lower() == 'stop':
                print("Stopped labeling.")
                updated_df = v2_saving_function(csv_read_pd,full_save_csv_path)
                raise SystemExit  # Exit gracefully
            
            try:
                num = int(inp) # convert to number (corresponding to label)
                if 0 <= num <= 11:  # Check if the number is in the valid range
                    clear_output(wait=True) # get rid of image once you've labelled it
                    csv_read_pd.loc[idx, '2nd_label'] = num ## as working in a df, we over-write the correct cell
                    break  # Exit the loop if the input is valid
                else:
                    print("Error: Label must be between 0 and 11.")
            except ValueError: # if we don't input a number -> stop labelling, so will break
                print("not words, what are you doing???")
        
updated_df = v2_saving_function(csv_read_pd,full_save_csv_path)

2ds_10000_8_dif.csv saved!


## hide extra stuff

In [7]:
csv_read_pd['2nd_label'].value_counts().get(1.0, 0)
#csv_read_pd['2nd_label'].count()
#df.to_csv(csv_file_list[0], index=False) 

2ds_10000_6.csv saved!


In [15]:
## copy newly labelled csv to github too
git_path = '/home/users/esree/CNN_basics/save_progress/'
shutil.copy(full_save_csv_path, git_path)

'/home/users/esree/CNN_basics/save_progress/2ds_10000_8_dif.csv'

# csv overwrite current label

In [5]:
og_csv = '2ds_10000.csv' ## original file
new_csv = '2ds_10000_8_dif.csv' ## new labels - in second column, but not all are second labelled
og_df = pd.read_csv(folder_path+og_csv)
new_df = pd.read_csv(folder_path+new_csv)

In [15]:
## combining og labels + new labels - replace old labels
length_of_data = len(og_df['image_name'])
name_lst = []
label_lst = []

for idx in range(int(length_of_data)):
    row = csv_read_pd.loc[idx]
    image_name = row.iloc[0]
    og_label = row.iloc[1]
    new_label = row.iloc[2]
    
    name_lst.append(image_name)
    if not pd.isna(new_label):
        label_lst.append(new_label)
    else:
        label_lst.append(og_label)

df_dic = {'image_name':name_lst,'number_label':label_lst}
new_df = pd.DataFrame(df_dic)
new_df

,image_name,number_label
0,1697685_23ch1,1.0
1,669294_20ch0,1.0
2,601343_19ch0,1.0
3,18058_26ch1,1.0
4,1261837_25ch0,3.0
...,...,...
9995,387342_02ch1,1.0
9996,676765_19ch0,1.0
9997,159619_06ch1,1.0
9998,2003638_23ch0,1.0


In [18]:
new_df['number_label'].count()
saving_csv_name = '2ds_10000_new.csv'
#saved_df = v2_saving_function(new_df,folder_path+saving_csv_name)

2ds_10000_new.csv saved!


# group images together to download them

In [44]:
folder_path = '/gws/nopw/j04/dcmex/users/ezriab/image_labelling/2ds_10000_sample/'
csv_file_name = '2ds_10000_new.csv'
csv_read_pd = pd.read_csv(folder_path+csv_file_name)

category_dic = {'1.0':'CP','2.0':'FA','3.0':'Co','4.0':'HPC','6.0':'CBC','7.0':'CA','8.0':'CC','9.0':'WD','0.0':'DIF','11.0':'unknown'}
category_dic_extra = {} # adding in path to loc 
for key in category_dic:
    var_name = category_dic[key]
    individ_cat_path = folder_path+var_name+'/'
    category_dic_extra[key] = [var_name,individ_cat_path]

'''
## make bunch of directories with appropriate names
category_names_lst = [category_dic[key] for key in category_dic] ## ayy make a list of category names from our dictionary values # newlist = [expression for item in iterable if condition == True]

# make each name a folder
for cat_name in category_names_lst:
    path = folder_path+cat_name+'/'
    if not os.path.exists(path):
        os.makedirs(path)
        print(f"{cat_name} created successfully!")
    else:
        print(f"{cat_name} already exists.")
'''


length_of_data = len(csv_read_pd['image_name'])
for idx in range(int(length_of_data/1000)):
    row = csv_read_pd.loc[idx]
    image_name = row.iloc[0]
    label = row.iloc[1]
    og_loc = folder_path+image_name+'.png'
    new_loc = category_dic_extra[str(label)][1]
    
    shutil.copy(og_loc, new_loc)

print('copying images finished')

# zip up the folders with images in
for key in category_dic_extra:
    cat_folder = category_dic_extra[key][0]
    folder_path_to_zip = category_dic_extra[key][1][:-1]
    shutil.make_archive(folder_path_to_zip, 'zip',folder_path_to_zip) # as we are saving with same name but with .zip
    print(f'{cat_folder} zipped!')

CP
/gws/nopw/j04/dcmex/users/ezriab/image_labelling/2ds_10000_sample/CP
FA
/gws/nopw/j04/dcmex/users/ezriab/image_labelling/2ds_10000_sample/FA
Co
/gws/nopw/j04/dcmex/users/ezriab/image_labelling/2ds_10000_sample/Co
HPC
/gws/nopw/j04/dcmex/users/ezriab/image_labelling/2ds_10000_sample/HPC
CBC
/gws/nopw/j04/dcmex/users/ezriab/image_labelling/2ds_10000_sample/CBC
CA
/gws/nopw/j04/dcmex/users/ezriab/image_labelling/2ds_10000_sample/CA
CC
/gws/nopw/j04/dcmex/users/ezriab/image_labelling/2ds_10000_sample/CC
WD
/gws/nopw/j04/dcmex/users/ezriab/image_labelling/2ds_10000_sample/WD
DIF
/gws/nopw/j04/dcmex/users/ezriab/image_labelling/2ds_10000_sample/DIF
unknown
/gws/nopw/j04/dcmex/users/ezriab/image_labelling/2ds_10000_sample/unknown


In [29]:
category_names

['CP', 'FA', 'Co', 'HPC', 'CBC', 'CA', 'CC', 'WD', 'DIF', 'unknown']

In [40]:
csv_read_pd[:10]

,image_name,number_label
0,1697685_23ch1,1.0
1,669294_20ch0,1.0
2,601343_19ch0,1.0
3,18058_26ch1,1.0
4,1261837_25ch0,3.0
5,484599_25ch0,2.0
6,1435287_23ch0,1.0
7,1710083_20ch0,1.0
8,2586247_25ch1,1.0
9,299315_20ch0,1.0


# extract relevant particle stats of ones labelled

In [48]:
path_2ds_stats = '/gws/nopw/j04/dcmex/users/ezriab/processed_stats/2ds/all_2ds.csv'
path_labelled = '/gws/nopw/j04/dcmex/users/ezriab/image_labelling/2ds_10000_sample/2ds_10000_new.csv'

full_ds_stats = pd.read_csv(path_2ds_stats)
image_labelled_df = pd.read_csv(path_labelled)

images_labelled_lst = list(image_labelled_df['image_name'])

labelled_image_stats = full_ds_stats[full_ds_stats['name'].isin(images_labelled_lst)]


## need to add new column - with the label on
# not sure why 2 extra rows on df more than have been labelled??

In [55]:
labelled_image_stats['name'].count()

10002

In [54]:
len(images_labelled_lst)

10000